In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

C:\Users\Saurav\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\Users\Saurav\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)


In [3]:
user_data = pd.read_table('/users/Saurav/documents/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv',
                          header = None, nrows = 2e7,
                          names = ['users', 'musicbrainz-artist-id', 'artist-name', 'plays'],
                          usecols = ['users', 'artist-name', 'plays'])

python
xyz
6
7.0
13.0
This isn't flying, this is falling with style!


In [4]:
user_profiles = pd.read_table('/users/Saurav/documents/lastfm-dataset-360K/usersha1-profile.tsv',
                          header = None,
                          names = ['users', 'gender', 'age', 'country', 'signup'],
                          usecols = ['users', 'country'])


In [5]:
user_data.head()

,users,artist-name,plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706


In [6]:
user_profiles.head()

,users,country
0,00000c289a1829a808ac09c00daf10bc3c4e223b,Germany
1,00001411dc427966b17297bf4d69e7e193135d89,Canada
2,00004d2ac9316e22dc007ab2243d6fcb239e707d,Germany
3,000063d3fe1cf2ba248b9e3c3f0334845a27a6bf,Mexico
4,00007a47085b9aab8af55f52ec8846ac479ac4fe,United States


In [7]:
if user_data['artist-name'].isnull().sum() > 0:
    user_data = user_data.dropna(axis = 0, subset = ['artist-name'])

In [8]:
artist_plays = (user_data.
     groupby(by = ['artist-name'])['plays'].
     sum().
     reset_index().
     rename(columns = {'plays': 'total_artist_plays'})
     [['artist-name', 'total_artist_plays']]
    )

In [9]:
artist_plays.head()

,artist-name,total_artist_plays
0,04)],6
1,2,1606
2,58725ab=>,23
3,80lİ yillarin tÜrkÇe sÖzlÜ aŞk Şarkilari,70
4,amy winehouse,23


In [10]:
user_data_with_artist_plays = user_data.merge(artist_plays, left_on = 'artist-name', right_on = 'artist-name', how = 'left')

In [11]:
user_data_with_artist_plays.head()
del user_data

In [12]:
artist_plays['total_artist_plays'].describe()

count     292363.000
mean       12907.022
std       185981.631
min            1.000
25%           53.000
50%          208.000
75%         1048.000
max     30466827.000
Name: total_artist_plays, dtype: float64

In [13]:
artist_plays['total_artist_plays'].quantile(np.arange(.9, 1, .01))

0.900     6137.800
0.910     7409.420
0.920     9102.040
0.930    11474.660
0.940    14898.000
0.950    19964.500
0.960    28420.120
0.970    43541.420
0.980    79403.560
0.990   198483.660
Name: total_artist_plays, dtype: float64

In [14]:
popularity_threshold = 40000
user_data_popular_artists = user_data_with_artist_plays.query('total_artist_plays >= @popularity_threshold')

In [15]:
user_data_popular_artists.head()

,users,artist-name,plays,total_artist_plays
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099,3704875
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897,180391
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717,410725
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706,90498
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691,13547741


In [16]:
combined = user_data_popular_artists.merge(user_profiles, left_on = 'users', right_on = 'users', how = 'left')
usa_data = combined.query('country == \'United States\'')
#usa_data = combined.query('country == \'United States\'|country == \'United Kingdom\'country == \'India\'')

In [17]:
usa_data.head()
del combined
del user_data_popular_artists
del user_data_with_artist_plays
del artist_plays
del user_profiles

In [18]:
if not usa_data[usa_data.duplicated(['users', 'artist-name'])].empty:
    initial_rows = usa_data.shape[0]

    print('Initial dataframe shape {0}'.format(usa_data.shape))
    usa_data = usa_data.drop_duplicates(['users', 'artist-name'])
    current_rows = usa_data.shape[0]
    print('New dataframe shape {0}'.format(usa_data.shape))
    print('Removed {0} rows'.format(initial_rows - current_rows))


Initial dataframe shape (2788019, 5)
New dataframe shape (2788013, 5)
Removed 6 rows


In [19]:
wide_artist_data = usa_data.pivot(index = 'artist-name', columns = 'users', values = 'plays').fillna(0)
#wide_artist_data_sparse = csr_matrix(wide_artist_data.values)

In [20]:
wide_artist_data_zero_one = wide_artist_data.apply(np.sign)
del wide_artist_data

In [21]:
#wide_artist_data_zero_one = wide_artist_data.apply(np.sign)
wide_artist_data_zero_one_sparse = csr_matrix(wide_artist_data_zero_one.values)

In [22]:
del wide_artist_data_zero_one_sparse

In [1]:
'''from scipy import sparse
'''
import pickle

fn = '/users/Saurav/documents/lastfm-dataset-360K/one.pkl'

#pickle.dump(wide_artist_data_zero_one, open(fn, 'wb'),protocol=4)
#sparse.save_npz('/users/Saurav/documents/lastfm-dataset-360K/lastfm_sparse_artist_matrix_binary.npz', wide_artist_data_zero_one_sparse)

In [2]:
from sklearn.neighbors import NearestNeighbors
import pickle
one_model=pickle.load(open(fn,'rb'))

C:\Users\Saurav\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\Users\Saurav\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
#model_nn_binary = NearestNeighbors(metric='cosine', algorithm='brute')
#model_nn_binary.fit(wide_artist_data_zero_one_sparse)
#model_nn_binary.fit('/users/Saurav/documents/lastfm-dataset-360K/lastfm_sparse_artist_matrix_binary.npz')
filename = '/users/Saurav/documents/lastfm-dataset-360K/finalized_model.pkl'
#pickle.dump(model_nn_binary, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))

In [4]:
'''
#DON'T USE THIS
model_nn_binary = NearestNeighbors(metric='cosine', algorithm='brute')
model_nn_binary.fit('/users/Saurav/documents/lastfm-dataset-360K/lastfm_sparse_artist_matrix_binary.npz')
'''

"\n#DON'T USE THIS\nmodel_nn_binary = NearestNeighbors(metric='cosine', algorithm='brute')\nmodel_nn_binary.fit('/users/Saurav/documents/lastfm-dataset-360K/lastfm_sparse_artist_matrix_binary.npz')\n"

In [5]:
'''
distances, indices = model_nn_binary.kneighbors(wide_artist_data_zero_one.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)

for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations with binary play data for {0}:\n'.format(wide_artist_data_zero_one.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, wide_artist_data_zero_one.index[indices.flatten()[i]], distances.flatten()[i]))
'''

"\ndistances, indices = model_nn_binary.kneighbors(wide_artist_data_zero_one.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)\n\nfor i in range(0, len(distances.flatten())):\n    if i == 0:\n        print('Recommendations with binary play data for {0}:\n'.format(wide_artist_data_zero_one.index[query_index]))\n    else:\n        print('{0}: {1}, with distance of {2}:'.format(i, wide_artist_data_zero_one.index[indices.flatten()[i]], distances.flatten()[i]))\n"

In [6]:
from fuzzywuzzy import fuzz

def print_artist_recommendations(query_artist, artist_plays_matrix, knn_model, k):
    """
    Inputs:
    query_artist: query artist name
    artist_plays_matrix: artist play count dataframe (not the sparse one, the pandas dataframe)
    knn_model: our previously fitted sklearn knn model
    k: the number of nearest neighbors.
    
    Prints: Artist recommendations for the query artist
    Returns: None
    """
    query_index = None
    ratio_tuples = []
    
    for i in artist_plays_matrix.index:
        ratio = fuzz.ratio(i.lower(), query_artist.lower())
        if ratio >= 75:
            current_query_index = artist_plays_matrix.index.tolist().index(i)
            ratio_tuples.append((i, ratio, current_query_index))
    
    print('Possible matches: {0}\n'.format([(x[0], x[1]) for x in ratio_tuples]))
    
    try:
        query_index = max(ratio_tuples, key = lambda x: x[1])[2] # get the index of the best artist match in the data
    except:
        print('Your artist didn\'t match any artists in the data. Try again')
        return None
    #print(artist_plays_matrix.iloc[query_index, :].values.reshape(1, -1))
    #print(k+1)
    distances, indices = knn_model.kneighbors(artist_plays_matrix.iloc[query_index, :].values.reshape(1, -1), n_neighbors = k + 1)
    
    for i in range(0, len(distances.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(artist_plays_matrix.index[query_index]))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, artist_plays_matrix.index[indices.flatten()[i]], distances.flatten()[i]))

    return None

In [8]:
#print_artist_recommendations('Eminem', wide_artist_data_zero_one, model_nn_binary, k = 10)
#print_artist_recommendations('Foo fighters', wide_artist_data_zero_one, loaded_model, k = 10)
print_artist_recommendations("disturbed", one_model, loaded_model, k = 10)

Possible matches: [('disturbed', 100)]

Recommendations for disturbed:

1: godsmack, with distance of 0.6492414978699145:
2: koЯn, with distance of 0.6599469766291561:
3: breaking benjamin, with distance of 0.6779740352004124:
4: slipknot, with distance of 0.6833986991923874:
5: linkin park, with distance of 0.6955731054139238:
6: system of a down, with distance of 0.6983363988585012:
7: three days grace, with distance of 0.703818585788144:
8: seether, with distance of 0.7132592597903307:
9: mudvayne, with distance of 0.7220875035627858:
10: papa roach, with distance of 0.7273468590413305:
